In [33]:
import zipfile
import os 
import pandas as pd
import shutil

In [ ]:
with zipfile.Zipfile("./이름_생년_성별_10000.zip", "r") as f:
    f.extractall('./data/')

In [12]:
gender = os.listdir("./data")[0].split("_")[-1][0] 

'3'

In [18]:
file

'┴ñ┐╣┴╓_19880710_2xxxxxx.txt'

In [35]:
male_list = []
female_list = list()

if not os.path.isdir("./남성"):
    os.mkdir("./남성")
if os.path.isdir("./여성")  == False:
    os.mkdir("./여성")
    
for file in os.listdir("./data"):
    gender = file.split("_")[-1][0]
    #if gender == '1' or gender == '3':
    #    print(gender)
    if gender in ['1', '3']:
        #print("남성", end= " ,")
        with open(f"./data/{file}", 'r', encoding='utf-8') as f:
            male_list.append(['남성', f.read().replace(",","")])
        shutil.move(f"./data/{file}", f"./남성/{file}")
    else:
        #print("여성", end=' ,')
        with open(f"./data/{file}", 'r', encoding='utf-8') as f:
            female_list.append(['여성', f.read().replace(",","")])
        shutil.move(f"./data/{file}", f"./여성/{file}")

In [36]:
male_df = pd.DataFrame(male_list, columns=['성별', '금액'])
female_df = pd.DataFrame(female_list, columns=['성별', '금액'])

In [41]:
df = pd.concat([male_df, female_df])

In [47]:
df['금액'] = df['금액'].astype(int)

In [52]:
df['금액2'] = list(map(int, df['금액']))

In [58]:
df.groupby(['성별'])['금액'].mean()

성별
남성    1.190018e+07
여성    1.280237e+07
Name: 금액, dtype: float64

In [63]:
df.groupby(['성별'])[['금액']].mean().loc['여성']

금액    1.280237e+07
Name: 여성, dtype: float64

In [65]:
df.groupby(['성별'])[['금액']].mean().reset_index().loc[1]

성별                 여성
금액    12802368.483323
Name: 1, dtype: object

In [66]:
df.groupby(['성별'])[['금액']].mean()

,금액
성별,
남성,1.190018e+07
여성,1.280237e+07


In [67]:
df.groupby(['성별'])[['금액']].mean().iloc[1]

금액    1.280237e+07
Name: 여성, dtype: float64

In [61]:
df.groupby(['성별'], as_index=False)[['금액']].mean()

,성별,금액
0,남성,1.190018e+07
1,여성,1.280237e+07


In [70]:
# agg -> aggregate
# avg -> mean 
df.groupby(['성별'], as_index=False)[['금액']].agg(['mean','median', 'var', 'std', 'count'])

성별            금액                                           
               mean   median           var           std count
0  남성  1.190018e+07  90329.0  6.400252e+14  2.529872e+07  4993
1  여성  1.280237e+07  96714.0  6.834308e+14  2.614251e+07  5007

In [78]:
import pickle
with open("./kbo_detail.pkl", "rb") as f:
    kbo = pickle.load(f)

In [74]:
data = {'df' : df ,
 'kbo' : kbo}

In [76]:
import pickle
with open("./tmp.pkl", "wb") as f:
    pickle.dump(data, f)

In [84]:
kbo['초등'] = kbo['경력'].apply(lambda x : x.split("-")[0])

In [89]:
kbo.groupby(['초등'])['선수명'].count().sort_values()[::-1].head(10)

초등
부산수영초    19
가동초      14
양정초      14
본리초      13
광주대성초    13
성동초      13
송정동초     12
광주수창초    12
화곡초      11
광주서석초    11
Name: 선수명, dtype: int64

In [96]:
kbo.groupby(['초등'])['선수명'].count().sort_values(ascending=False) 

초등
부산수영초           19
가동초             14
양정초             14
광주대성초           13
본리초             13
                ..
부산대연초(남구리틀)      1
부산대연초(사상구리틀)     1
부산안락초(해운대리틀)     1
부산인지초(해운대리틀)     1
서울온곡초(광진구리틀)     1
Name: 선수명, Length: 350, dtype: int64

In [98]:
kbo.value_counts(['초등'], normalize=True)

초등          
부산수영초           0.022512
가동초             0.016588
양정초             0.016588
광주대성초           0.015403
본리초             0.015403
                  ...   
부산대연초(남구리틀)     0.001185
부산대연초(사상구리틀)    0.001185
부산안락초(해운대리틀)    0.001185
부산인지초(해운대리틀)    0.001185
서울온곡초(광진구리틀)    0.001185
Name: proportion, Length: 350, dtype: float64

In [103]:
from datetime import date, datetime 

In [107]:
datetime.strptime("1997년 04월 03일", "%Y년 %m월 %d일")

datetime.datetime(1997, 4, 3, 0, 0)

In [110]:
kbo['생년월일'] = kbo['생년월일'].apply(lambda x : datetime.strptime(x.strip(), "%Y년 %m월 %d일"))

In [118]:
kbo['생존일'] = kbo['생년월일'].apply(lambda x : (datetime.now() - x).days)

In [123]:
kbo.sort_values(by=['생존일', '선수명'], ascending=[False, True]).head(3)

,선수명,등번호,생년월일,포지션,신장/체중,경력,입단 계약금,연봉,지명순위,입단년도,team,초등,생존일
251,추신수,No.17,1982-07-13,외야수(좌투좌타),180cm/97kg,부산수영초-부산중-부산고-SK,,3000만원,07 해외진출선수 특별지명,00시애틀,SSG 랜더스,부산수영초,15245
718,오승환,No.21,1982-07-15,투수(우투우타),178cm/93kg,도신초-우신중-경기고-단국대-삼성-한신-세인트루이스-토론토-콜로라도,18000만원,40000만원,05 삼성 2차 1라운드 5순위,05삼성,삼성 라이온즈,도신초,15243
4,김강민,No.9,1982-09-13,외야수(우투우타),182cm/87kg,본리초-대구중-경북고-SK-SSG,11000만원,11000만원,01 SK 2차 2라운드 18순위,01SK,한화 이글스,본리초,15183


In [126]:
kbo['나이'] = kbo['생년월일'].apply(lambda x : datetime.now().year - x.year)

In [129]:
kbo.team.unique()

array(['한화 이글스', 'KIA 타이거즈', 'SSG 랜더스', 'NC 다이노스', 'LG 트윈스', '키움 히어로즈',
       '고양 히어로즈', '두산 베어스', '롯데 자이언츠', '삼성 라이온즈', 'KT 위즈'], dtype=object)

In [135]:
kbo.loc[kbo.team == "고양 히어로즈", 'team']  = '키움 히어로즈'

In [138]:
kbo.groupby(['team'])['나이'].agg(['mean', 'median', 'var'])

,mean,median,var
team,,,
KIA 타이거즈,25.988095,25.0,25.698652
KT 위즈,26.275862,25.0,28.504411
LG 트윈스,26.013514,25.0,23.027212
NC 다이노스,25.112360,24.0,20.214505
SSG 랜더스,26.259259,25.0,34.419444
두산 베어스,25.678571,25.0,23.064114
롯데 자이언츠,26.152941,25.0,30.131092
삼성 라이온즈,26.329412,25.0,32.961625
키움 히어로즈,24.719512,24.0,25.191960


In [141]:
kbo[kbo.team == "삼성 라이온즈"].value_counts("나이").head(5)

나이
23    10
25     9
21     8
20     7
22     6
Name: count, dtype: int64

In [146]:
kbo[kbo.team == "키움 히어로즈"].value_counts("나이", normalize=True).head(5).cumsum()

나이
20    0.195122
19    0.292683
24    0.390244
23    0.475610
25    0.548780
Name: proportion, dtype: float64

In [147]:
kbo

,선수명,등번호,생년월일,포지션,신장/체중,경력,입단 계약금,연봉,지명순위,입단년도,team,초등,생존일,나이
0,강재민,No.,1997-04-03,투수(우언우타),180cm/89kg,양덕초-마산중-용마고-단국대,8000만원,14500만원,20 한화 2차 4라운드 38순위,20한화,한화 이글스,양덕초,9867,27
1,권광민,No.17,1997-12-12,외야수(좌투좌타),189cm/102kg,서울청구초-홍은중-장충고,,3300만원,22 한화 2차 5라운드 41순위,16시카고컵스,한화 이글스,서울청구초,9614,27
2,권동욱,No.02,2001-01-08,외야수(우투우타),173cm/85kg,가동초-휘문중-배명고-동국대,,3000만원,24 한화 육성선수,,한화 이글스,가동초,8491,23
3,권현,No.111,2005-02-23,외야수(우투우타),182cm/88kg,사당초-언북중-장충고,3000만원,3000만원,24 한화 10라운드 91순위,,한화 이글스,사당초,6984,19
4,김강민,No.9,1982-09-13,외야수(우투우타),182cm/87kg,본리초-대구중-경북고-SK-SSG,11000만원,11000만원,01 SK 2차 2라운드 18순위,01SK,한화 이글스,본리초,15183,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839,한민우,No.96,1999-04-27,투수(좌투좌타),177cm/82kg,동일중앙초-경남중-경남고,1000만원,3000만원,23 KT 육성선수,,KT 위즈,동일중앙초,9113,25
840,한차현,No.59,1998-11-30,투수(우투우타),180cm/80kg,사능초(남양주리틀)-청원중-포항제철고-성균관대,10000만원,3100만원,21 KT 2차 2라운드 15순위,21KT,KT 위즈,사능초(남양주리틀),9261,26
841,홍현빈,No.31,1997-08-29,외야수(우투좌타),174cm/70kg,수원신곡초-매송중-유신고-KT-상무,9000만원,4500만원,17 KT 2차 3라운드 21순위,17KT,KT 위즈,수원신곡초,9719,27
842,황의준,No.103,2002-04-06,외야수(우투좌타),185cm/95kg,칠성초-경복중-경북고-수성대,4000만원,3000만원,23 KT 8라운드 80순위,,KT 위즈,칠성초,8038,22


In [166]:
import requests 
from bs4 import BeautifulSoup as BS 

r= requests.get("http://gs25.gsretail.com/gscvs/ko/store-services/locations")

bs = BS(r.text)

#<form id="CSRFForm" action="/gscvs/ko/store-services/locations" method="post"><input type="hidden" name="CSRFToken" value="406690e8-0340-4931-b372-4b5519577c3c" />
bs.find("input", {'name': "CSRFToken"})

<input name="CSRFToken" type="hidden" value="90492b27-2a5c-46dc-8c91-d21c7b9b7c6e"/>

In [207]:
3476 * 5 / 100

173.8

In [208]:
payload = {"pageNum": "1",
"pageSize": "5",
"searchShopName": "",
"searchSido": "",
"searchGugun": "",
"searchDong": "",
"searchType": "",
"searchTypeService": "0",
"searchTypeToto": "0",
"searchTypeCafe25": "0",
"searchTypeInstant": "0",
"searchTypeDrug": "0",
"searchTypeSelf25": "0",
"searchTypePost": "0",
"searchTypeATM": "0",
"searchTypeWithdrawal": "0",
"searchTypeTaxrefund": "0",
"searchTypeSmartAtm": "0",
"searchTypeSelfCookingUtensils": "0",
"searchTypeDeliveryService": "0",
"searchTypeParcelService": "0",
"searchTypePotatoes": "0",
"searchTypeCardiacDefi": "0",
"searchTypeFishShapedBun": "0",}
total = []
with requests.Session() as s:
    r = s.get("http://gs25.gsretail.com/gscvs/ko/store-services/locations")
    bs = BS(r.text)
    token = bs.find("input", {'name': "CSRFToken"})['value']
    #print(bs.find("input", {'name': "CSRFToken"}))
    target = f"http://gs25.gsretail.com/gscvs/ko/store-services/locationList?CSRFToken={token}"
    for x in range(1,3):
        if x % 10 == 0: print(x)
        payload['pageNum'] = x 
        r2 = s.post(target, data=payload)
        total.append( pd.DataFrame(eval(r2.json())['results']))

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170


In [210]:
gs25 = pd.concat(total)

In [212]:
with open("./gs25.pkl", "wb") as f:
    pickle.dump(gs25, f)

In [ ]:
with open("./gs25.pkl", "rb") as f:
    gs25 = pickle.load(f)

In [214]:
gs25.shopCode.unique().size

17378

In [220]:
gs25.reset_index(drop=True, inplace=True)

In [221]:
gs25

,offeringService,shopCode,longs,address,shopName,lat
0,"[toto, cafe25, post, withdrawal, parcel_service]",V3414,37.51826292,"경기 부천시 오정구 석천로345, 300동 114호(삼정동 365)",GS252테크노파크점,126.76388845
1,"[cafe25, drug, post, atm, withdrawal, smart_at...",VX815,38.00414846,"강원 양양군 현북면 동해대로1242, 38선휴게소 (잔교리 41-10, 38휴게소)",GS2538선휴게소점,128.73283627
2,"[post, parcel_service]",VQ037,37.52831065,"서울 양천구 목동서로159-1 (목1동 917-1, CBS방송국1층)",GS25CBS점,126.87486792
3,"[cafe25, potatoes, fish_shaped_bun]",V7W76,36.44932895,"대전 대덕구 대덕대로1447번길39, 1층 (문평동 140)",GS25CJ대전HUB점,127.40708036
4,NaN,V6Z47,37.69488283,"경기 고양시 일산동구 성현로615 (사리현동 151-1, CJ대한통운)",GS25CJ덕양SUB점,126.84958684
...,...,...,...,...,...,...
17374,NaN,V1880,37.56433789,"서울 성동구 왕십리로지하 374, 207-02호 (하왕십리동 946-14, 상왕십리역)",S2상왕십리역점,127.02961475
17375,NaN,V1882,37.50287602,"서울 동작구 현충로257, 431-02호 (동작동 327-1)",S4동작역점,126.9802623
17376,"[drug, post, withdrawal, tax, parcel_service]",VW993,37.59431984,"서울 동대문구 경희대로23, 문화복지시설 101호 (회기동 1-5, 전면플라자)",경희프라자점,127.05231488
17377,NaN,V1355,37.50208899,서울 강남구 논현로508 (역삼동 679),나만의냉장고,127.03678844
